In [ ]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import warnings

warnings.filterwarnings('ignore')

# --- 步驟 1: 獲取並準備數據 ---
# 使用 yfinance 獲取台積電(2330.TW)過去幾年的股價數據
ticker = "2330.TW"
df = yf.download(ticker, start="2020-01-01", end="2024-12-31")

print(f"成功獲取 {ticker} 的數據，共 {len(df)} 筆。")
print(df.head())
print(df.columns)
df.columns = df.columns.droplevel(0)
for col in df.columns:
    print(f'Column: "{col}"')
df.columns = list(df.columns)

# --- 步驟 2: 計算技術指標 (組合一) ---
# 1. 計算 RSI (相對強弱指標)
# 2. 計算 MACD (包含其柱狀圖 MACD Histogram)
# 我們將使用 pandas_ta 套件，它能方便地在 DataFrame 上計算指標
df.ta.rsi(length=14, append=True)
df.ta.macd(fast=12, slow=26, signal=9, append=True)

# 檢視一下計算出的欄位，RSI_14 是 RSI，MACDh_12_26_9 是 MACD Histogram
print("\n計算技術指標後的數據預覽：")
print(df.tail())


# --- 步驟 3: 定義預測目標 (Target) ---
# 目標：預測隔天 (t) 是漲還是跌
# 定義：如果 t 日的收盤價 > t-1 日的收盤價，則為 1 (漲)，否則為 0 (跌)
# 我們透過將收盤價向上移動一格來實現，shift(-1) 會將未來的數據移到當前行
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)

print("\n加入 Target 後的數據預覽：")
print(df[['Close', 'Close_t+1', 'Target']].head())


# --- 步驟 4: 建立特徵 (Features) 並進行延遲 (Lag) 處理 ---
# 這是最關鍵的一步，確保我們用 t-1 的資訊去預測 t
# 我們將剛才計算的指標延遲一期
df['RSI_lag1'] = df['RSI_14'].shift(1)
df['MACD_Hist_lag1'] = df['MACDh_12_26_9'].shift(1)

# 刪除因為計算指標和延遲所產生的 NaN/空值行
df.dropna(inplace=True)


# --- 步驟 5: 準備訓練與測試資料集 ---
# 特徵 X: 使用延遲後的指標
features = ['RSI_lag1', 'MACD_Hist_lag1']
X = df[features]

# 目標 y: 我們要預測的漲跌結果
y = df['Target']

# 將數據分割為訓練集 (80%) 和測試集 (20%)
# shuffle=False 確保數據按時間順序分割，這在時間序列中是好的做法
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print(f"\n數據準備完成：")
print(f"訓練集大小: {len(X_train)} 筆")
print(f"測試集大小: {len(X_test)} 筆")


# --- 步驟 6: 建立並訓練隨機森林模型 ---
# 隨機森林是一個強大且常用的分類模型
model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)

# 使用訓練集來訓練模型
model.fit(X_train, y_train)

print("\n模型訓練完成！")


# --- 步驟 7: 評估模型表現 ---
# 使用訓練好的模型對測試集進行預測
y_pred = model.predict(X_test)

# 計算並印出模型的準確率
accuracy = accuracy_score(y_test, y_pred)
print(f"\n模型在測試集上的準確率 (Accuracy): {accuracy:.4f}")

# 印出更詳細的分類報告 (包含精確率、召回率)
print("\n分類報告 (Classification Report):")
print(classification_report(y_test, y_pred, target_names=['跌 (Down)', '漲 (Up)']))

# 印出混淆矩陣 (Confusion Matrix)
#       預測為跌  預測為漲
# 實際為跌   TN        FP
# 實際為漲   FN        TP
print("\n混淆矩陣 (Confusion Matrix):")
print(pd.DataFrame(confusion_matrix(y_test, y_pred), 
                 index=['實際 跌', '實際 漲'], 
                 columns=['預測 跌', '預測 漲']))